# Make Bag-of-visual-words

* Load Image's descriptor for all images. 
* Find corresponding visual word index for each descriptor. 
* Save list of visual word index.

# For Oxford 5k dataset, You can use already provided visual words 

In [1]:
# Oxford 5k dataset provides already converted visual words. We could use this one
import os 
import pickle

oxf5k_visualword_dir = './data/word_oxc1_hesaff_sift_16M_1M'

filelist = os.listdir(oxf5k_visualword_dir)
filelist.sort()
# print(filelist)
# for parent_dir, _, files in os.walk(oxf5k_visualword_dir):
#     print(files)

bow_dict = {}
count_descriptor = 0
image_feature_count_info = []
for filename in filelist:
    filepath = os.path.join(oxf5k_visualword_dir, filename)
    image_name = filename.replace(".txt", "")    
    visual_words = []
    with open(filepath) as f:
        lines = list(map(lambda x: x.strip(), f.readlines()[2:])) # ignore first two lines        
        for l in lines:
            val = l.split(" ")
            visual_word_index = int(val[0])-1 # This data use 1 to 1,000,000. convert to zero-based so 0 to 999,999  
            visual_words.append(visual_word_index)
            # print('{} descriptor {}'.format(filename, l))
        count_descriptor = count_descriptor + len(lines)
    image_feature_count_info.append((image_name, len(visual_words)))
    bow_dict[image_name] = sorted(visual_words)
    # break
# print('bow_dict:', bow_dict)
print('count_descriptor:', count_descriptor)


pickle.dump(bow_dict, open('bow_dict_word_oxc1_hesaff_sift_16M_1M_pretrained.pkl', 'wb'))


count_descriptor: 16334970


# Or we could make a new one

## Requirements

* Image name and its associated 128d descriptors
* visual words assigner
    * For a given 128d descriptor, it tells index of visual words. 
    * After you ran k-means clustering, you could use it to get nearest centroid's id


In [ ]:
# For oxford 5k dataset, it may be possible to recover the "Image name and its associated 128d descriptors"
from utils.oxf5k_feature_reader import feature_reader

# Frist, read all 128d descriptor of image.
feature_bin_path = "./data/feature/feat_oxc1_hesaff_sift.bin"
all_features = feature_reader(feature_bin_path)
print('num of 128d descriptors in oxf5k: ', len(all_features))


In [ ]:
# Read the order of filenames related to above features. Refer README in http://www.robots.ox.ac.uk/~vgg/data/oxbuildings/README2.txt
feature_bin_filename_order = "./data/feature/order.txt"
with open(feature_bin_filename_order) as f:
    filenames = list(map(lambda x: x.strip(), f.readlines()))

# Check compatibility of name order
for idx, val in enumerate(image_feature_count_info):
    name_from_vw, _ = val
    if name_from_vw == filenames[idx]:
        continue
    else:
        print(idx, name_from_vw, filenames[idx])
print("compatibility check done")    

In [ ]:
import numpy as np
image_descriptor_dict = {} # key: image_name, value: 2d numpy array of shape (num_descriptor, dim_descriptor)
start_idx = 0
for image_name, num_descriptor in image_feature_count_info:
    val = np.array(all_features[start_idx:(start_idx+num_descriptor)], dtype=np.uint8)
    image_descriptor_dict[image_name] = val
    
    start_idx += num_descriptor
    # break
# print('image_descriptor_dict:', image_descriptor_dict)
with open('image_descriptor_dict_oxc1_hesaff_sift_16M.pkl', 'wb') as f:
    pickle.dump(image_descriptor_dict, f)

In [2]:
# Load image descriptor dictionary, and assign each descriptor to visual words
import pickle
with open('image_descriptor_dict_oxc1_hesaff_sift_16M.pkl', 'rb') as f:
    # key: image_name, value: 2d numpy array of shape (num_descriptor, dim_descriptor)
    image_descriptor_dict = pickle.load(f) 
print(len(image_descriptor_dict))

with open('kmeans.pkl', 'rb') as f:
    kmeans = pickle.load(f)
    
# For PQ-kmeans clustering, we first convert query to PQ codes. 
with open('encoder.pkl', 'rb') as f:
    encoder = pickle.load(f)

    
bow_dict = {}
for image_name in image_descriptor_dict.keys():
    descriptors = image_descriptor_dict[image_name]    
    data_points_pqcode = encoder.transform(descriptors)
    visual_words = kmeans.fit_predict(data_points_pqcode)
    bow_dict[image_name] = sorted(visual_words)
    sorted(clustered)
    print(clustered)
    break
pickle.dump(bow_dict, open('bow_dict_word_oxc1_hesaff_sift_16M_1M_handmade.pkl', 'wb'))



5062
